In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# Load environment variables
load_dotenv()

# Define the Agent class with detailed prompts for realistic, task-specific estimations
class Agent:
    def __init__(self, name, system_prompt, llm):
        self.name = name
        self.system_prompt = system_prompt
        self.llm = llm
        self.chat_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self.system_prompt),
            HumanMessagePromptTemplate.from_template("{conversation_history}\n{input}")
        ])
        self.chain = LLMChain(
            llm=self.llm,
            prompt=self.chat_prompt,
            verbose=False
        )

    def respond(self, input_text, conversation_history):
        response = self.chain.predict(
            input=input_text,
            conversation_history=conversation_history
        )
        return response

# Initialize LLM with the API key
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    openai_api_key="ADD_YOUR_OPENAI_KEY"
)

# Define agents with enhanced prompts for diverse, real-world estimations
product_owner = Agent(
    name="Product_Owner",
    system_prompt=(
        "You are the Product Owner. In each sprint planning session, your role is to ensure that the team fully understands the priority and nuances of each requirement. "
        "Clarify requirements in detail, highlight areas of complexity, and provide feedback based on customer expectations. Aim to set the team up for accurate estimations."
    ),
    llm=llm
)

scrum_master = Agent(
    name="Scrum_Master",
    system_prompt=(
        "You are the Scrum Master. Your role is to keep the team aligned with Scrum principles and ensure smooth progress through each sprint. "
        "Facilitate planning sessions, daily standups, reviews, and retrospectives, helping the team address blockers and promote efficiency."
    ),
    llm=llm
)

agents = {
    "Requirement_Engineer": Agent(
        name="Requirement_Engineer",
        system_prompt=(
            "You are the Requirement Engineer. Analyze and document each user story in detail, identifying individual requirements specific to a healthcare assistant chatbot. "
            "Consider unique challenges such as data privacy, medical information accuracy, and user-friendly interactions. "
            "Estimate work based on the number of distinct requirements for each story, following these steps:\n"
            "- Step 1: Work = estimate the total number of unique requirements, considering medical complexities.\n"
            "- Step 2: Choose a productivity rate that reflects the complexity of healthcare requirements.\n"
            "- Step 3: Calculate effort as Effort = Work / Productivity rate."
        ),
        llm=llm
    ),
    "System_Engineer": Agent(
        name="System_Engineer",
        system_prompt=(
            "You are the System Engineer. Design the technical architecture for each user story, creating a comprehensive design document for a healthcare chatbot. "
            "Consider elements like secure data handling, API integration for health resources, and UX for sensitive information. "
            "Estimate work based on the total pages needed, following these steps:\n"
            "- Step 1: Work = estimate the total pages required in the design document based on healthcare-specific needs.\n"
            "- Step 2: Set a productivity rate that reflects technical complexity in healthcare.\n"
            "- Step 3: Calculate effort as Effort = Work / Productivity rate."
        ),
        llm=llm
    ),
    "Software_Developer": Agent(
        name="Software_Developer",
        system_prompt=(
            "You are the Software Developer. Based on the design, develop software components for the chatbot, considering healthcare-specific needs such as data validation, medical information retrieval, and patient privacy compliance. "
            "Estimate work based on lines of code (SLOC) for each feature, and follow these steps:\n"
            "- Step 1: Work = estimate the total SLOC required for each feature, considering healthcare data handling and validation.\n"
            "- Step 2: Choose a productivity rate based on the complexity of each software component.\n"
            "- Step 3: Calculate effort as Effort = Work / Productivity rate."
        ),
        llm=llm
    ),
    "Test_Engineer": Agent(
        name="Test_Engineer",
        system_prompt=(
            "You are the Test Engineer. Develop and execute test cases for each user story, ensuring the chatbot’s accuracy and reliability, especially for healthcare data and recommendations. "
            "Plan for edge cases like incorrect symptom input, privacy tests, and regulatory compliance. "
            "Follow these calculation steps:\n"
            "- Step 1: Work = estimate total test cases for each story, considering healthcare requirements.\n"
            "- Step 2: Set a productivity rate appropriate to the complexity and importance of healthcare testing.\n"
            "- Step 3: Calculate effort as Effort = Work / Productivity rate."
        ),
        llm=llm
    ),
    "Documentation_Engineer": Agent(
        name="Documentation_Engineer",
        system_prompt=(
            "You are the Documentation Engineer. Create clear and user-friendly documentation for end-users and healthcare staff, covering aspects like symptom input, privacy settings, and chatbot features. "
            "Estimate work based on the number of documentation pages needed, considering healthcare accessibility and readability standards. "
            "Follow these steps:\n"
            "- Step 1: Work = estimate the total pages required for comprehensive documentation.\n"
            "- Step 2: Set a productivity rate that reflects the level of detail and accessibility needed for healthcare users.\n"
            "- Step 3: Calculate effort as Effort = Work / Productivity rate."
        ),
        llm=llm
    )
}

# Define sprints and user stories specific to healthcare chatbot functionalities
sprints = [
    {"Sprint 1": ["User Registration and Secure Login"]},
    {"Sprint 2": ["Symptom Checker with User Input Validation", "Medicine Information Retrieval"]},
    {"Sprint 3": ["Dietary Advice and Nutritional Suggestions", "General Health Tips and Preventive Advice"]},
    {"Sprint 4": ["User Profile Management with Health History", "Feedback Collection", "Logout and Data Privacy Assurance"]}
]

# Initialize conversation history
conversation_history = ""
initial_message = (
    "We are developing a virtual healthcare assistant chatbot. "
    "The chatbot should help users with secure logins, symptom checks, medicine information, dietary advice, and provide health tips, while prioritizing data privacy and accuracy."
)
conversation_history += f"Product_Owner: {initial_message}\n"

# Refined Scrum ceremony prompts for a healthcare assistant chatbot project
scrum_prompts = {
    "Sprint_Planning": {
        "Product_Owner": (
            "During Sprint Planning, clarify priorities and specific details for each user story: {user_stories}. "
            "Highlight aspects that may need additional attention, such as data security or medical accuracy, ensuring realistic task estimates."
        ),
        "Requirement_Engineer": (
            "For each story, identify requirements specific to healthcare interactions, such as data security, accuracy in symptom checking, and user-friendly interfaces. "
            "Estimate work and calculate effort, ensuring each requirement aligns with healthcare standards."
        ),
        "System_Engineer": (
            "Plan the design tasks for each story, accounting for secure data handling, health API integration, and patient data privacy. "
            "Estimate work and calculate effort based on the technical scope needed for healthcare-grade design."
        ),
        "Software_Developer": (
            "Define coding requirements for each user story, focusing on reliable, secure code for healthcare functionalities. "
            "Estimate work and calculate effort based on complexity and the need for accuracy in health data processing."
        ),
        "Test_Engineer": (
            "Plan test cases for each story, ensuring accuracy and reliability in healthcare data handling and user privacy. "
            "Estimate work and calculate effort based on the complexity of tests needed for compliance with healthcare standards."
        ),
        "Documentation_Engineer": (
            "Plan documentation tasks for each story, focusing on clear, accessible instructions and healthcare-specific use cases. "
            "Estimate work and calculate effort with attention to readability and regulatory compliance."
        )
    },
    "Daily_Standup": {
        "Scrum_Master": (
            "Each day, team members should report:\n"
            "- What they worked on yesterday\n- What they plan to work on today\n"
            "- Any blockers or issues they face, especially related to healthcare data handling or regulatory needs. "
            "Assist in addressing blockers to ensure steady progress."
        )
    },
    "Sprint_Review": {
        "Product_Owner": (
            "During the Sprint Review, assess the completed work for each story, paying special attention to healthcare requirements. "
            "Provide feedback on accuracy, data handling, and usability, suggesting adjustments for the next sprint if necessary."
        )
    },
    "Sprint_Retrospective": {
        "Scrum_Master": (
            "Reflect on the sprint’s outcomes:\n- What went well, particularly regarding healthcare-specific challenges?\n"
            "- What could be improved in terms of accuracy, privacy, or user experience?\n"
            "- Identify action items for continuous improvement in healthcare project standards."
        )
    }
}

# Function for running each Scrum sprint with detailed steps
def run_sprint(sprint_name, sprint_stories):
    global conversation_history
    print(f"--- {sprint_name} ---")
    print(f"User Stories: {', '.join(sprint_stories)}\n")

    # Sprint Planning
    scrum_master_response = scrum_master.respond(
        f"Start Sprint Planning for {sprint_name}. Coordinate with Product Owner for story details.",
        conversation_history
    )
    print(f"Scrum_Master:\n{scrum_master_response}\n")
    conversation_history += f"Scrum_Master: {scrum_master_response}\n"

    po_response = product_owner.respond(
        scrum_prompts["Sprint_Planning"]["Product_Owner"].format(user_stories=", ".join(sprint_stories)),
        conversation_history
    )
    print(f"Product_Owner:\n{po_response}\n")
    conversation_history += f"Product_Owner: {po_response}\n"

    # Assign each role-specific planning task
    for role, agent in agents.items():
        task_message = scrum_prompts["Sprint_Planning"][role]
        response = agent.respond(task_message, conversation_history)
        print(f"{agent.name}:\n{response}\n")
        conversation_history += f"{agent.name}: {response}\n"

    # Daily Standups (simulated for simplicity)
    print("\n--- Daily Standups ---")
    for role, agent in agents.items():
        standup_message = "Provide a daily update on progress."
        response = agent.respond(standup_message, conversation_history)
        print(f"{agent.name} Standup:\n{response}\n")
        conversation_history += f"{agent.name} Standup: {response}\n"

    # Sprint Review
    print("\n--- Sprint Review ---")
    review_message = scrum_prompts["Sprint_Review"]["Product_Owner"]
    review_response = product_owner.respond(review_message, conversation_history)
    print(f"Product_Owner Feedback:\n{review_response}\n")
    conversation_history += f"Product_Owner: {review_response}\n"

    # Sprint Retrospective
    print("\n--- Sprint Retrospective ---")
    retrospective_message = scrum_prompts["Sprint_Retrospective"]["Scrum_Master"]
    retrospective_response = scrum_master.respond(retrospective_message, conversation_history)
    print(f"Scrum_Master:\n{retrospective_response}\n")
    conversation_history += f"Scrum_Master: {retrospective_response}\n"

# Loop through each sprint
for sprint in sprints:
    sprint_name = list(sprint.keys())[0]
    sprint_stories = sprint[sprint_name]
    
    # Start the sprint process
    run_sprint(sprint_name, sprint_stories)

    # Option to continue or stop after each sprint
    continue_sprints = input("Would you like to continue to the next sprint or stop? (Type 'continue' or 'stop'): ").strip().lower()
    if continue_sprints == 'stop':
        print("Exiting the process.")
        break

print("All sprints completed or exited.")


C:\Users\ashar\AppData\Local\Temp\ipykernel_21860\1546563494.py:38: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(
C:\Users\ashar\AppData\Local\Temp\ipykernel_21860\1546563494.py:24: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  self.chain = LLMChain(


--- Sprint 1 ---
User Stories: User Registration and Secure Login

Scrum_Master:
Absolutely, let's get started with Sprint Planning for Sprint 1. Here's how we can structure our session:

### Sprint Planning Agenda

1. **Review the Product Backlog**: 
   - Collaborate with the Product Owner to review the items in the backlog that are relevant for the first sprint. 

2. **Define Sprint Goal**: 
   - Establish a clear and concise sprint goal that aligns with the overall project vision for the virtual healthcare assistant chatbot.

3. **Select Backlog Items**: 
   - Identify which user stories will be included in this sprint. We’ll want to focus on the highest priority items that can be realistically completed in the sprint timeframe.

4. **Clarify User Stories**:
   - We'll need to ask the Product Owner for details on each selected story, including acceptance criteria and any dependencies.

5. **Estimate Stories**:
   - Use a method such as Planning Poker to estimate the effort required 